In [1]:
import os
import torch
import torch.nn as nn
import torchtext
from tensorboardX import SummaryWriter
import random
import numpy as np
from pytorch_pretrained_bert import BertModel
from pytorch_pretrained_bert.optimization import BertAdam
from tqdm import tqdm
from model import *
from utils import AverageMeter

%load_ext autoreload
%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

03/12/2019 20:39:23 - INFO - summarizer.preprocessing.cleaner -   'pattern' package not found; tag filters are not available for English


In [2]:
class Config:
    def __init__(self):
        self.lr = 1e-5
        self.warmup_proportion = 0.1
        self.epochs = 5
        self.max_seq_length = 128
        self.gradient_accumulation_steps = 16
        self.log_dir = './logs'
        self.model_name = 'bert_finetune'
        
        self.train_bert_path = './data/train_data_bert_128.pt'
        self.dev_bert_path = './data/dev_data_bert_128.pt'        

        self.bert_type = 'base'
        self.bert_path = './bert-base-uncased/'
        self.bert_dim = 768
        
        self.dropout = 0.2
        self.seed = 1023

        
config = Config()
device = torch.device("cuda:0")


In [3]:
random.seed(config.seed)
np.random.seed(config.seed)
torch.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)

In [4]:
save_path = config.model_name  + '_lr_'+ str(config.lr)+ '_dropout_' + str(config.dropout) \
            + '_maxSeqLen_' + str(config.max_seq_length) + '_bertType_' + config.bert_type + '__epochs_' + str(config.epochs)

save_path = os.path.join(config.log_dir, save_path)   
print(save_path)
config.save_path = save_path

./logs/bert_finetune_lr_1e-05_dropout_0.2_maxSeqLen_128_bertType_base__epochs_5


In [5]:
if not os.path.exists(config.save_path):
    os.makedirs(config.save_path)
writer = SummaryWriter(config.save_path)

In [6]:
class BertForWiki(nn.Module):
    
    def __init__(self, config, device):
        super(BertForWiki, self).__init__()
        
        self.device = device
        self.bert_model = BertModel.from_pretrained(config.bert_path)

        self.pq_w_attn = CoAttention(hidden_dims=config.bert_dim, att_type=0, dropout=config.dropout)
        
        # only use max row pooling for simplify
        self.pooling_1 = SelfAttention(config.bert_dim, config.bert_dim, config.dropout)
        self.pooling_2 = SelfAttention(config.bert_dim, config.bert_dim, config.dropout)

        self.pooling_3 = SelfAttention(config.bert_dim, config.bert_dim, config.dropout)
        
        self.linear_last = BilinearSeqAttn(config.bert_dim, config.bert_dim, dropout=config.dropout)

        self.to(device)
        
    def forward(self, choices, passages, question, max_batch =32):
        device = self.device
        
        batch = tuple( t.to(device) for t in choices)
        input_ids, input_mask, segment_ids  = batch
        all_encoder_layers, _ = self.bert_model(input_ids, segment_ids, input_mask)
        c_emb = all_encoder_layers[-1]

        
        batch = tuple( t.to(device) for t in passages)
        input_ids, input_mask, segment_ids  = batch
        
        if input_ids.size(0) > max_batch:
            input_ids = input_ids[:max_batch]
            input_mask = input_mask[:max_batch]
            segment_ids = segment_ids[:max_batch]
            
        all_encoder_layers, _ = self.bert_model(input_ids, segment_ids, input_mask)
        p_emb = all_encoder_layers[-1]

        batch = tuple( t.to(device) for t in question)
        input_ids, input_mask, segment_ids  = batch
        all_encoder_layers, _ = self.bert_model(input_ids, segment_ids, input_mask)
        q_emb = all_encoder_layers[-1]
        
        q_emb_p = q_emb.expand(p_emb.size(0), q_emb.size(1), q_emb.size(2))
        p_attn_out, qp_attn_out = self.pq_w_attn(p_emb, q_emb_p)


        # p_len * bert_hidden
        p_summarys = self.pooling_1(p_attn_out)
        
        # 1 * bert_hidden
        p_final = self.pooling_2(p_summarys.unsqueeze(0)) 

        # 1 * c_len * hidden
        c_final = self.pooling_3(c_emb).unsqueeze(0)
        
        # 1 * c_len
        score = self.linear_last(c_final, p_final)
        
        return score

In [7]:
model = BertForWiki(config, device)

In [8]:
train_data = torch.load(config.train_bert_path)
dev_data = torch.load(config.dev_bert_path)

In [9]:
dev_data[0]

{'choices': (tensor([[  101,  5118,   102,     0,     0],
          [  101, 11068,  1997,  9192,   102],
          [  101,  2605,   102,     0,     0],
          [  101,  2446, 11078,   102,     0],
          [  101,  2446,  3400,   102,     0],
          [  101,  2762,   102,     0,     0],
          [  101,  4151,  3142,  3400,   102],
          [  101,  3304,   102,     0,     0],
          [  101,  2900,   102,     0,     0],
          [  101,  2983,  1997,  2605,   102],
          [  101,  2983,  1997, 13019,   102],
          [  101,  2642,  3304,   102,     0],
          [  101,  3142,  3400,   102,     0],
          [  101,  3607,   102,     0,     0],
          [  101, 13019,   102,     0,     0],
          [  101,  2142,  2983,   102,     0],
          [  101, 20695,  3072,   102,     0],
          [  101,  2088,   102,     0,     0]]), tensor([[1, 1, 1, 0, 0],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 0, 0],
          [1, 1, 1, 1, 0],
          [1, 1, 1, 1, 0],
        

In [10]:
param_optimizer = list(model.named_parameters())

# hack to remove pooler, which is not used
# thus it produce None grad that break apex
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

t_total = len(train_data) // config.gradient_accumulation_steps * config.epochs
print(t_total)

optimizer = BertAdam(optimizer_grouped_parameters,
                             lr=config.lr,
                             warmup=config.warmup_proportion,
                             t_total=t_total)
criterion = torch.nn.CrossEntropyLoss()

13665


In [11]:
def val(data_iter, model, criterion):
    losses = AverageMeter()
    acces = AverageMeter()
    model.eval()
    for idx, item in enumerate(tqdm(data_iter)):
        choices, passages, question = item['choices'], item['passages'], item['question']
        label = item['label'].to(device)
        with torch.no_grad():
            score = model(choices, passages, question)
                    
        loss = criterion(score, label)
        losses.update(loss.item())
        
        pred = score.argmax(1)
        acc = pred.eq(label).sum().item()  / pred.size(0)
        acces.update(acc)
    return losses.avg, acces.avg

def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x

In [12]:
global_step = 0

for ep in range(config.epochs):
    random.shuffle(train_data)
    model.train()
    losses = AverageMeter()
    acces = AverageMeter()    

    for step, item in enumerate(tqdm(train_data)):
        choices, passages, question = item['choices'], item['passages'], item['question']
        label = item['label'].to(device)
        score = model(choices, passages, question)
        
        loss = criterion(score, label)
        loss = loss / config.gradient_accumulation_steps

        losses.update(loss.item())
        pred = score.argmax(1)
        acc = pred.eq(label).sum().item()  / pred.size(0)
        acces.update(acc)
        loss.backward()
        if(step+1) % config.gradient_accumulation_steps == 0:
            '''
            lr_this_step = config.lr * warmup_linear(global_step/t_total, config.warmup_proportion)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_this_step
            '''
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1  
        if global_step % 20 == 0:                            
            writer.add_scalar('train_loss', losses.avg, global_step)
            writer.add_scalar('train_acc', acces.avg, global_step)
            print(f'step:{global_step}, train_loss:{losses.avg}, acc:{acces.avg}')
            
    print('eval on dev set')
    val_loss, val_acc = val(dev_data, model, criterion)
    print(f'{val_loss}, {val_acc}')
    writer.add_scalar('val_loss', val_loss, ep)
    writer.add_scalar('val_acc', val_acc, ep)

  0%|          | 0/43738 [00:00<?, ?it/s]/home/user2/Multi_QA/model.py:184: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  alphas = self.softmax(alphas)  # (bsz, sent_len)
  0%|          | 1/43738 [00:00<5:57:20,  2.04it/s]

step:0, train_loss:0.29659658670425415, acc:0.0


  0%|          | 2/43738 [00:01<6:32:33,  1.86it/s]

step:0, train_loss:0.29428306221961975, acc:0.0


  0%|          | 3/43738 [00:01<5:27:35,  2.23it/s]

...

step:13660, train_loss:0.059652727936777374, acc:0.6718961134141035


  0%|          | 1/5129 [00:00<16:48,  5.08it/s]

eval on dev set


100%|██████████| 5129/5129 [13:40<00:00,  6.39it/s]

1.293597065150915, 0.5800350945603432


In [16]:
acces.avg

0.671795692532809

In [14]:
item = train_data[0]
criterion = torch.nn.CrossEntropyLoss()
choices, passages, question = item['choices'], item['passages'], item['question']
label = item['label'].to(device)
score = model(choices, passages, question)

loss = criterion(score, label)
loss = loss / config.gradient_accumulation_steps

loss.backward()
print(loss)

/home/user2/Multi_QA/model.py:184: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  alphas = self.softmax(alphas)  # (bsz, sent_len)


tensor(5.9813e-05, device='cuda:0', grad_fn=<DivBackward0>)


In [15]:
inputs = []

def hook(module, input, output):
    inputs.append(input)
    
